In [1]:
import sys
sys.path.append(r'E:\jupyter-notebook\packages')# modify
from waveform import waveform
import numpy as np
import scipy
from scipy import signal
import matplotlib.pyplot as plt
from scipy.fftpack import fft,ifft
from scipy.fft import fftshift

amp=np.load(r"E:\CDEX\0vbb\Code\AE_par\E_Trap.npy")
par=np.load(r"E:\CDEX\0vbb\Code\AE_par\Ecali_par.npy")
A=np.load(r"E:\CDEX\0vbb\Code\AE_par\A_bio_6_8.npy")
E=par[0]+amp*par[1]
AOE=A/E

In [2]:
filename="G:/CDEX/DATA/\
20200331_t3_Th_BEGe_2026_6us_0.5_5_2111_0.5_10_Difout10_FADC_RAW_Data_%d.bin"#modify
savename="G:/CDEX/wave_par/"

In [3]:
###cut around current peak (+-L)
L=128
waveiterator=waveform(filename,range(5))
A0,amp0=waveiterator.iterate()
E0=par[0]+amp0*par[1]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [7]:
Dep_index=np.argwhere(abs(E0-1592)<3)
Sep_index=np.argwhere(abs(E0-2103)<3)
print(Dep_index)

[[  232     1]
 [  586     1]
 [  637     0]
 [ 1623     1]
 [ 2979     1]
 [ 6385     0]
 [ 6419     2]
 [ 6569     1]
 [ 7192     1]
 [ 7260     0]
 [ 7389     1]
 [ 7871     0]
 [ 7989     2]
 [ 8374     1]
 [10301     2]
 [11738     0]
 [12128     1]
 [12491     0]
 [13076     0]
 [13829     1]
 [17118     1]
 [17276     2]
 [17303     0]
 [17305     2]
 [18134     2]
 [18302     0]
 [18974     2]
 [19548     2]
 [20266     2]
 [20766     1]
 [21642     2]
 [21649     1]
 [21701     1]
 [22223     0]
 [22388     0]
 [22706     0]
 [22767     0]
 [22941     0]
 [23524     0]
 [23669     0]
 [25196     1]
 [25204     0]
 [25305     0]
 [25522     1]
 [26419     1]
 [26930     1]
 [28337     1]
 [28576     0]
 [29014     2]
 [29091     1]
 [29449     0]
 [29526     1]
 [30219     0]
 [30951     1]
 [30970     2]
 [31075     2]
 [32798     1]
 [34030     0]
 [34070     2]
 [34120     1]
 [34194     1]
 [34375     2]
 [35132     0]
 [35215     2]
 [35288     2]
 [36011     0]
 [37571   

In [8]:
Depwave=np.zeros((len(Dep_index),2*L))
k=0
for i in Dep_index:
    filenum=int(i[0]/10000)
    eventnum=i[0]%10000
    waveiterator.read_event(filename%filenum,eventnum)
    waveiterator.cut_around_risep(0.5,2*L)
    #waveiterator.interpolate(5,)
    waveiterator.Partial(5)
    Depwave[k,:]=waveiterator.processedw[:]
    k+=1

In [ ]:
####plot average current pulse

ave_wave=np.zeros((1,Depwave.shape[1]))
wave_num=0
fig, ax = plt.subplots(1,1,figsize=(8, 6),dpi=1200)
for i in range(Dep_index.shape[0]):
    if(np.max(Depwave[i,0:110])>20 or np.max(Depwave[i,150:256])>15):
        continue
    
    ax.plot(Depwave[i,:],linewidth=0.5)
    wave_num+=1
    ave_wave+=Depwave[i,:]
ave_wave[0,:]=ave_wave[0,:]/wave_num
ax.plot(ave_wave[0,:],label='Average current pulse',color='r')
ax.set(xlabel='time (10ns)',ylabel='amplitude')
ax.legend()

plt.subplots_adjust(left=0.10, bottom=0.10, right=0.97, top=0.97,
                wspace=0.15, hspace=0.25)
#fig.savefig(r"E:\CDEX\0vbb\Figure\A\ave_current.png")

In [11]:
import matplotlib.gridspec as gridspec

fig = plt.figure(tight_layout=True,figsize=[10,8],dpi=1200)
gs = gridspec.GridSpec(3, 2)

ave_wave=np.zeros((1,Depwave.shape[1]))
wave_num=0

ax = fig.add_subplot(gs[0, :])
for i in range(Dep_index.shape[0]):
    if(np.max(Depwave[i,0:110])>20 or np.max(Depwave[i,150:256])>15):
        continue
    
    ax.plot(Depwave[i,:],linewidth=0.5)
    wave_num+=1
    ave_wave+=Depwave[i,:]
ave_wave[0,:]=ave_wave[0,:]/wave_num
ax.plot(ave_wave[0,:],label='Average current pulse',color='r')
ax.set(xlabel='time (10ns)',ylabel='amplitude')
ax.legend()

ax = fig.add_subplot(gs[1, 0])
ax.plot(ave_wave[0,:],label='average current pulse')
ax.legend()

Filter=fft(ave_wave[0,:]).conjugate()
ax = fig.add_subplot(gs[1, 1])
ax.plot(abs(ifft(Filter)),label='filter hit response')
ax.legend()

ax = fig.add_subplot(gs[2, 0])
ax.plot(Depwave[i,:],label='current pulse before filter')
ax.legend()

ax = fig.add_subplot(gs[2, 1])
Input=fft(Depwave[i,:])
Output=fftshift(ifft(Input[0:256]*Filter[0:256]))
ax.plot(abs(Output[:]),label='filted pusle after filter')
ax.legend()

fig.align_labels()  # same as fig.align_xlabels(); fig.align_ylabels()

plt.show()

fig.savefig(r"E:\CDEX\0vbb\Figure\A\mfilter_process.png")

In [ ]:
###interpolate 5 point 
##filter
i=5
fig, ax = plt.subplots(2,2,figsize=(12, 9),dpi=1200)

ax[0,0].plot(ave_wave[0,:],label='average current pulse')

Filter=fft(ave_wave[0,:]).conjugate()
ax[0,1].plot(abs(ifft(Filter)),label='filter hit response')
ax[1,0].plot(Depwave[i,:],label='current pulse before filter')
Input=fft(Depwave[i,:])
Output=fftshift(ifft(Input[0:256]*Filter[0:256]))
ax[1,1].plot(abs(Output[:]),label='filted pusle after filter')

plt.subplots_adjust(left=0.10, bottom=0.10, right=0.97, top=0.97,
                wspace=0.15, hspace=0.15)
ax[0,0].legend()
ax[1,0].legend()
ax[0,1].legend()
ax[1,1].legend()
plt.show()
fig.savefig(r"E:\CDEX\0vbb\Figure\A\filter_process.png")
#plt.plot(abs(ifft(Input)))
#plt.plot(abs(Input))
###FFT


In [ ]:
np.save(r"E:\CDEX\0vbb\Code\AE_par\ave_wave",ave_wave)
np.save(r"E:\CDEX\0vbb\Code\AE_par\Depwave",Depwave)
np.save(r"E:\CDEX\0vbb\Code\AE_par\mfilter",Filter)